# Lab 4: Momentum II

In the last lab we explored how to backtest decile portfolio style trading strategies. In this lab we will explore how to backtest portfolios that are optimized each period to maximize alpha while minimizing variance. 

## Imports

In [1]:
import sf_quant.data as sfd
import sf_quant.optimizer as sfo
import sf_quant.backtester as sfb
import sf_quant.performance as sfp
import polars as pl
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

/home/stiten/Documents/sf-quant-labs/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-22 12:49:24,582	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Data

We import the necessary data for you here. We will be doing a backtest from 2023-01-01 to 2024-01-31. However since our signal takes 1 year to compute, we will really only be backtesting 1 month of data.

In [2]:
start = dt.date(2023, 1, 1)
end = dt.date(2024, 1, 31)

columns = [
    'date',
    'barrid',
    'ticker',
    'price',
    'return',
    'specific_risk',
    'predicted_beta'
]

data = sfd.load_assets(
    start=start,
    end=end,
    in_universe=True,
    columns=columns
)

data

date,barrid,ticker,price,return,specific_risk,predicted_beta
date,str,str,f64,f64,f64,f64
2023-01-03,"""USA06Z1""","""MDXG""",2.97,6.8345,47.65898,1.43155
2023-01-04,"""USA06Z1""","""MDXG""",3.0,1.0101,47.539847,1.428854
2023-01-05,"""USA06Z1""","""MDXG""",3.08,2.6667,47.755957,1.37346
2023-01-06,"""USA06Z1""","""MDXG""",3.21,4.2208,48.110135,1.389274
2023-01-09,"""USA06Z1""","""MDXG""",3.28,2.1807,48.612414,1.348687
…,…,…,…,…,…,…
2024-01-25,"""USBPM41""","""WS""",29.38,-0.3054,50.968819,1.408389
2024-01-26,"""USBPM41""","""WS""",29.58,0.6807,50.922137,1.40225
2024-01-29,"""USBPM41""","""WS""",30.28,2.3665,50.93332,1.395143


## Compute the Momentum Signal

## Instructions

- Compute momentum for each security and date as the rolling 230 day return (you can just use log returns here).
- Shift the momentum signal 22 days. This will results in the 11 month return from t-12 to t-2.

In [3]:
def task_compute_momentum(data: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the t_12 to t_2 momentum signal for each secrutiy and date combination.
    
    Args:
        data (pl.DataFrame): Data frame containing date, barrid, price, and return columns.
    
    Returns:
        pl.DataFrame: Data frame with columns date, barrid, price, return, and momentum columns.
    """
    log_returns = data.with_columns(
        pl.col('return').truediv(100).log1p().alias('log_return')
    )
    momentum = ( log_returns.sort('date', 'barrid').with_columns(
            pl.col('log_return').rolling_sum(window_size=230).over('barrid').alias('momentum')
        )
        .with_columns(
            pl.col('momentum').shift(22).over('barrid').alias('momentum')
        )
    )
    
    return momentum

momentum = task_compute_momentum(data)

momentum

date,barrid,ticker,price,return,specific_risk,predicted_beta,log_return,momentum
date,str,str,f64,f64,f64,f64,f64,f64
2023-01-03,"""USA06Z1""","""MDXG""",2.97,6.8345,47.65898,1.43155,0.066111,null
2023-01-03,"""USA0C11""","""FBIN""",57.98,1.5234,17.242483,1.258299,0.015119,null
2023-01-03,"""USA0H41""","""INBK""",24.49,0.8649,38.364938,1.146046,0.008612,null
2023-01-03,"""USA0SY1""","""UI""",270.37,-1.1553,32.807458,1.022137,-0.01162,null
2023-01-03,"""USA11I1""","""EBAY""",42.15,1.6397,20.777432,1.215919,0.016264,null
…,…,…,…,…,…,…,…,…
2024-01-31,"""USBPGY1""","""BIRK""",46.39,-2.3368,31.858302,1.232728,-0.023645,null
2024-01-31,"""USBPID1""","""NATL""",22.39,-2.4401,31.44603,1.174775,-0.024704,null
2024-01-31,"""USBPJV1""","""NLOP""",24.78,-1.9391,63.437399,1.455797,-0.019581,null


## Compute the Alphas

In order to make our momentum signal usable in our optimizer we will use a predetermined Information Coefficient of 0.05 and the forecasted idiosyncratic risk provided by Barra to convert our signal into alpha forecasts.

### Instructions
- For each date z-score the momentum signal across all asssets cross sectionally and call this `score`.
- Using the `specific_risk` column compute the alphas as `0.05` * `score` * `specific_risk`.
- Note: Make sure to divide `specific_risk` by 100 to put it in decimal space. 

In [4]:
def task_compute_alphas(momentum: pl.DataFrame) -> pl.DataFrame:
    """ 
    Compute the alphas for each security and date combo.

    Args:
        momentum (pl.DataFrame): Data frame containing barrid, date, specific_risk, and momentum columns.
    
    Returns:
        pl.DataFrame: Data frame containing barrid, date, specific_risk, momentum, score, and alpha columns.
    """
    clean_momentum = momentum.with_columns(pl.col('specific_risk').truediv(100).fill_null(strategy='forward').over('barrid'))

    scores_df = clean_momentum.with_columns(
            ((pl.col('momentum') - pl.col('momentum').mean().over('date')) / (pl.col('momentum').std().over('date'))).alias('score'),
        )
    alphas = scores_df.with_columns(
            (0.05 * pl.col('score') * pl.col('specific_risk').truediv(100)).alias('alpha')
        )
    return alphas

alphas = task_compute_alphas(momentum)

alphas

date,barrid,ticker,price,return,specific_risk,predicted_beta,log_return,momentum,score,alpha
date,str,str,f64,f64,f64,f64,f64,f64,f64,f64
2023-01-03,"""USA06Z1""","""MDXG""",2.97,6.8345,0.4765898,1.43155,0.066111,null,null,null
2023-01-03,"""USA0C11""","""FBIN""",57.98,1.5234,0.172425,1.258299,0.015119,null,null,null
2023-01-03,"""USA0H41""","""INBK""",24.49,0.8649,0.383649,1.146046,0.008612,null,null,null
2023-01-03,"""USA0SY1""","""UI""",270.37,-1.1553,0.328075,1.022137,-0.01162,null,null,null
2023-01-03,"""USA11I1""","""EBAY""",42.15,1.6397,0.207774,1.215919,0.016264,null,null,null
…,…,…,…,…,…,…,…,…,…,…
2024-01-31,"""USBPGY1""","""BIRK""",46.39,-2.3368,0.318583,1.232728,-0.023645,null,null,null
2024-01-31,"""USBPID1""","""NATL""",22.39,-2.4401,0.3144603,1.174775,-0.024704,null,null,null
2024-01-31,"""USBPJV1""","""NLOP""",24.78,-1.9391,0.634374,1.455797,-0.019581,null,null,null


In [5]:
def task_price_filter(alphas: pl.DataFrame) -> pl.DataFrame:
    """
    Filter the universe to lagged price greater than 5 and non-null alpha.
    
    Args:
        alphas (pl.DataFrame): Data frame containing barrid, date, specific_risk, momentum, score, and alpha columns.
    Returns:
        pl.DataFrame: Data frame containing barrid, date, specific_risk, momentum, score, and alpha columns.
    """
    alphas_filtered = (alphas
                        .sort('barrid', 'date')
                        .with_columns(pl.col('price').shift(1).over('barrid').alias('price_lag'))
                        .filter(
            pl.col('price_lag') > 5, 
            pl.col('alpha').is_not_null()
       )
       .sort('barrid', 'date')
    )
    return alphas_filtered

price_filter = task_price_filter(alphas)

price_filter

date,barrid,ticker,price,return,specific_risk,predicted_beta,log_return,momentum,score,alpha,price_lag
date,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-01-03,"""USA06Z1""","""MDXG""",7.775,-1.2071,0.498276,1.092049,-0.012144,1.027828,2.291715,0.000571,7.87
2024-01-04,"""USA06Z1""","""MDXG""",7.76,-0.1929,0.497463,1.098982,-0.001931,0.989638,2.186411,0.000544,7.775
2024-01-05,"""USA06Z1""","""MDXG""",7.8,0.5155,0.495355,1.063097,0.005142,0.979587,2.186923,0.000542,7.76
2024-01-08,"""USA06Z1""","""MDXG""",8.22,5.3846,0.494352,1.085788,0.052446,0.936866,2.069079,0.000511,7.8
2024-01-09,"""USA06Z1""","""MDXG""",8.03,-2.3114,0.494685,1.182189,-0.023385,0.896796,2.054735,0.000508,8.22
…,…,…,…,…,…,…,…,…,…,…,…
2024-01-29,"""USBONP1""","""MBC""",14.65,2.2331,0.3174,1.346623,0.022085,0.526594,1.172817,0.000186,14.33
2024-01-30,"""USBONP1""","""MBC""",14.59,-0.4096,0.317128,1.334121,-0.004104,0.502256,1.122952,0.000178,14.65
2024-01-31,"""USBONP1""","""MBC""",14.07,-3.5641,0.316861,1.356568,-0.036292,0.521707,1.151499,0.000182,14.59


## Backtest

Now that we have our alphas we will compute the MVO portfolios for each date in our sample.

### Instructions
- Use the `FullInvestment`, `LongOnly`, `NoBuyingOnMargin`, and `UnitBeta` constraints.
- For each unique date in the `price_filter` data frame find the optimal weights using `sf_quant.optimizer.mve_optimizer`.
- Note: for the `UnitBeta` constraint to work you will need to provide the predicted betas to the optimizer in each itteration.
- Hint: the optimizer assumes that your alpha vector and covariance matrix are both sorted the same way.
- Hint: use a gamma of 10.

In [6]:
def task_backtest(price_filter: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the optimal portfolio weights for each day in our sample.
    
    Args:
        price_filter (pl.DataFrame): Data frame containing barrid, date, specific_risk, momentum, score, and alpha columns.
    Returns:
        pl.DataFrame: Data frame containing barrid, date, and weight columns.
    """

#     constraints = [sfo.LongOnly(), sfo.FullInvestment(), sfo.NoBuyingOnMargin(), sfo.UnitBeta()]

#     daily_portfolio = []
#     for date_ in tqdm.tqdm(price_filter.select(pl.col("date")).unique().
#         sort(by="date").to_series().to_list()):
#         date_df = price_filter.filter(pl.col("date") == date_).sort("barrid")
#         alphas =  date_df.select(pl.col("alpha")).to_numpy().flatten()
#         betas = date_df.select(pl.col("predicted_beta")).to_numpy().flatten()
#         barrids = date_df.select(pl.col("barrid")).to_series().to_list()
#         cov_mat = sfd.construct_covariance_matrix(date_=date_, barrids=barrids).drop('barrid').to_numpy()
#         weights = sfo.mve_optimizer(
#             ids=barrids,
#             alphas=alphas,
#             covariance_matrix=cov_mat,
#             constraints=constraints,
#             gamma=10,
#             betas=betas
#         )

#         daily_portfolio.append(weights)

#     return pl.concat(daily_portfolio)

# weights = task_backtest(price_filter)

# weights

    dates = price_filter['date'].unique().sort().to_list()

    constraints = [
        sfo.FullInvestment(),
        sfo.LongOnly(),
        sfo.NoBuyingOnMargin(),
        sfo.UnitBeta()
    ]

    portfolio_list = []
    for date_ in tqdm.tqdm(dates, "Computing portfolios"):
        subset = (
            price_filter.filter(
                pl.col('date').eq(date_)
            )
            .sort('barrid')
        )
        barrids = subset['barrid'].to_list()
        alphas_np = subset['alpha'].to_numpy()
        betas_np = subset['predicted_beta'].to_numpy()

        cov_mat = sfd.construct_covariance_matrix(date_, barrids).drop('barrid').to_numpy()

        portfolio = sfo.mve_optimizer(
            ids=barrids,
            alphas=alphas_np,
            covariance_matrix=cov_mat,
            constraints=constraints,
            gamma=10,
            betas=betas_np
        )

        portfolio = portfolio.with_columns(
            pl.lit(date_).alias('date')
        )

        portfolio_list.append(portfolio)

    return pl.concat(portfolio_list)

weights = task_backtest(price_filter)

weights

Computing portfolios:   0%|          | 0/20 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (76,77) (77,76) (76,77) 

In [27]:
        
date_df = price_filter.filter(pl.col("date") == dt.date(2024, 1, 31)).sort("barrid")
alphas =  date_df.select(pl.col("alpha")).to_numpy().flatten()
betas = date_df.select(pl.col("predicted_beta")).to_numpy().flatten()
barrids = date_df.select(pl.col("barrid")).to_series().to_list()
len(betas)

2407

## Performance Analysis

Now that we have our optimal weights we will join the returns from our initial dataset. 

### Instructions
- Join the returns from `data` and compute the return and cumulative return of the portfolio using the optimal weights.
- Note: since our covariance matrix isn't lagged we will need to shift our returns forward. To do this use `.shift(-1)` by `barrid` and call it `fwd_return`.
- Chart the cumulative returns of the portfolio.

In [ ]:
def task_compute_returns(weights: pl.DataFrame, data: pl.DataFrame) -> pl.DataFrame:
    """ 
    Compute the optimal portfolio returns.

    Args:
        weights (pl.DataFrame): Data frame containing barrid, date, and weight columns.
        data (pl.DataFrame): Data frame containing barrid, date, and return columns

    Returns:
        pl.DataFrame: Data frame containing date, fwd_return, and cumulative_fwd_return_columns
    """
    merged = weights.join(
        data,
        on=['barrid', 'date'],
        how='left'
    ).with_columns(
        (pl.col('weight') * pl.col('return').truediv(100)).alias('contribution')
    )

    daily_returns = merged.groupby('date').agg(
        pl.col('contribution').sum().alias('fwd_return').shift(-1)
    ).sort('date').with_columns(
        (1 + pl.col('fwd_return')).cumprod().alias('cumulative_fwd_return')
    )

    return daily_returns         

returns = task_compute_returns(weights, data)

returns

In [ ]:
# TODO: Chart the cumulative returns of the portfolio.

## Benchmark Decomposition

You should find that our portfolio is up and to the right. But the question is how much of that is due to the market being up versus our signal being good. We will find out by joining the benchmark weights to our `weights` data frame and computing the active weights.

### Instructions

- Pull in the benchmark weights using `sf_quant.data.load_benchmark`.
- Join the benchmark weights to the optimal weights.
- Compute the active weights as `weight` - `weight_bmk` = `weight_act`
- Unpivot the weight columns and compute the forward return for each portfolio (total, benchmark, and active). 

In [ ]:
def task_return_decomposition(weights: pl.DataFrame, data: pl.DataFrame) -> pl.DataFrame:
    """ 
    Compute the forward returns for the total, benchmark, and active portfolios.

    Args:
        weights (pl.DataFrame): Data frame containing barrid, date, and weight columns.
        data (pl.DataFrame): Data frame containing barrid, date, and return columns

    Returns:
        pl.DataFrame: Data frame containing date, portfolio, fwd_return, and cumulative_fwd_return columns        
    """
    # TODO: Finish this function.
    pass

returns_decomp = task_return_decomposition(weights, data)

returns_decomp

In [ ]:
# TODO: Chart the cumulative returns of each portfolio
# HINT: Use seaborn.lineplot() with the attribute hue='portfolio'

In [ ]:
# TODO: Compute the annual average return, annual volatility, and annualized sharpe ratio for each portfolio.

## `sf_quant` Backtester Module

That was a lot of fun right? Just kidding. All of that code takes a lot of work. That's why we've implemented a backtester in the `sf_quant` package. Let's practice using it really quick and compare our results.

### Instructions
- Declare your constraints the same way you did previously.
- Use a gamma of 10.
- Find the optimal weights using the `sf_quant.backtester` module.
- Hint: use the `backtest_parallel()` module to run your backtest in parallel across all the cores on your machine.

In [ ]:
def task_backtest_sf(price_filter: pl.DataFrame) -> pl.DataFrame:
    """ 
    Compute the optimal portfolio weights using the `sf_quant` package.

    Args:
        price_filter (pl.DataFrame): Data frame containing barrid, date, specific_risk, momentum, score, and alpha columns.

    Returns:
        pl.DataFrame: Data frame containing barrid, date, and weight columns.
    """
    # TODO: Finish this function.
    pass

weights_sf = task_backtest_sf(price_filter)

weights_sf

## `sf_quant` Performance Package

It's also not a lot of fun to merge the returns dataset and do a full decomposition manually. You can do that with `sf_quant.performance` too.

### Instructions

- Compute the portfolio forward returns decomposition using the `generate_returns_from_weights` function.
- Chart the cumulative returns of the portfolios using the `generate_returns_chart` function.
- Generate the summary table using the `generate_summary_table` function.

In [ ]:
def task_return_decomposition_sf(weights_sf: pl.DataFrame) -> pl.DataFrame:
    """ 
    Compute the returns decomposition using the `sf_quant` package.

    Args:
        weights_sf (pl.DataFrame): Data frame containing date, barrid, and weight columns.

    Returns:
        pl.DataFrame: Data frame containing date, portfolio, and return (fwd_return) columns
    """
    # TODO: Finish this function.
    pass

returns_sf = task_return_decomposition_sf(weights_sf)

returns_sf

In [ ]:
# TODO: Generate the returns chart

In [ ]:
# TODO: Generate the summary table